[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/marmalux/Clientes_TelecomX/blob/main/Clientes_TelecomX.ipynb)


# Análisis de evasión de clientes en Telecom X

## Importación de datos

In [1]:
import numpy as np
import pandas as pd
import requests
import json

#importar de una API y pasar a json
url = 'https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/refs/heads/main/TelecomX_Data.json'
datos_api = requests.get(url)
datos_json=json.loads(datos_api.text)

In [15]:
# normalizar datos
datos_cliente = pd.json_normalize(datos_json)
datos_cliente.sample(5)

,customerID,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,phone.PhoneService,phone.MultipleLines,internet.InternetService,...,internet.OnlineBackup,internet.DeviceProtection,internet.TechSupport,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,account.Charges.Monthly,account.Charges.Total
400,0572-ZJKLT,No,Female,0,Yes,Yes,46,Yes,Yes,Fiber optic,...,No,Yes,Yes,No,Yes,Two year,No,Mailed check,99.65,4630.2
7021,9667-EQRXU,Yes,Female,1,No,No,30,No,No phone service,DSL,...,No,Yes,No,Yes,No,Month-to-month,Yes,Electronic check,40.30,1172.95
5344,7297-DVYGA,No,Female,0,No,No,51,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,One year,Yes,Bank transfer (automatic),19.85,996.95
4501,6169-PPETC,No,Male,0,Yes,Yes,35,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Bank transfer (automatic),20.50,759.35
6219,8559-CIZFV,Yes,Male,0,Yes,Yes,21,Yes,Yes,DSL,...,Yes,No,Yes,Yes,No,One year,No,Mailed check,77.50,1625


### Diccionario de datos

- `customerID`: número de identificación único de cada cliente
- `Churn`: si el cliente dejó o no la empresa
- `gender`: género (masculino y femenino)
- `SeniorCitizen`: información sobre si un cliente tiene o no una edad igual o mayor a 65 años
- `Partner`: si el cliente tiene o no una pareja
- `Dependents`: si el cliente tiene o no dependientes
- `tenure`: meses de contrato del cliente
- `PhoneService`: suscripción al servicio telefónico
- `MultipleLines`: suscripción a más de una línea telefónica
- `InternetService`: suscripción a un proveedor de internet
- `OnlineSecurity`: suscripción adicional de seguridad en línea
- `OnlineBackup`: suscripción adicional de respaldo en línea
- `DeviceProtection`: suscripción adicional de protección del dispositivo
- `TechSupport`: suscripción adicional de soporte técnico, menor tiempo de espera
- `StreamingTV`: suscripción de televisión por cable
- `StreamingMovies`: suscripción de streaming de películas
- `Contract`: tipo de contrato
- `PaperlessBilling`: si el cliente prefiere recibir la factura en línea
- `PaymentMethod`: forma de pago
- `Charges.Monthly`: total de todos los servicios del cliente por mes
- `Charges.Total`: total gastado por el cliente

Comprobación de datos

In [3]:
datos_cliente.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customerID                 7267 non-null   object 
 1   Churn                      7267 non-null   object 
 2   customer.gender            7267 non-null   object 
 3   customer.SeniorCitizen     7267 non-null   int64  
 4   customer.Partner           7267 non-null   object 
 5   customer.Dependents        7267 non-null   object 
 6   customer.tenure            7267 non-null   int64  
 7   phone.PhoneService         7267 non-null   object 
 8   phone.MultipleLines        7267 non-null   object 
 9   internet.InternetService   7267 non-null   object 
 10  internet.OnlineSecurity    7267 non-null   object 
 11  internet.OnlineBackup      7267 non-null   object 
 12  internet.DeviceProtection  7267 non-null   object 
 13  internet.TechSupport       7267 non-null   objec

In [16]:
# print(datos_cliente['Churn'].unique()) # en la columna Churn hay valores incoherentes

# datos_cliente['Churn'].str.strip()
(datos_cliente[(datos_cliente['Churn'] == '')].head())
# (datos_cliente[(datos_cliente['account.Charges.Total'] == ' ')].head())


,customerID,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,phone.PhoneService,phone.MultipleLines,internet.InternetService,...,internet.OnlineBackup,internet.DeviceProtection,internet.TechSupport,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,account.Charges.Monthly,account.Charges.Total
975,1371-DWPAZ,No,Female,0,Yes,Yes,0,No,No phone service,DSL,...,Yes,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),56.05,
1775,2520-SGTTA,No,Female,0,Yes,Yes,0,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.00,
1955,2775-SEFEE,No,Male,0,No,Yes,0,Yes,Yes,DSL,...,Yes,No,Yes,No,No,Two year,Yes,Bank transfer (automatic),61.90,
2075,2923-ARZLG,No,Male,0,Yes,Yes,0,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,One year,Yes,Mailed check,19.70,
2232,3115-CZMZD,No,Male,0,No,Yes,0,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.25,


Después de revisar, la columna *Churn* y la columna *account.Charges.Total* tienen valores inconsistentes, *Churn* faltan  valores de 'Yes' pues depués de analizar, hay gastos totales por lo que debe ser aun cliente de la empresa, y la columna de gastos totales *account.Charges.Total* tiene valores vacios no nulos, pero como son clientes que ya no estan en la empresa se puden dejar como 0 para poder cambiar el tipo de dato

In [17]:
# transformar columna Churn
datos_cliente['Churn'] = datos_cliente['Churn'].replace('',pd.NA).fillna('No')# se remplazan las celdas vacias a valores nulos y despues se rellenan esos valores con 'No'

#transformar columna 	account.Charges.Total
datos_cliente['account.Charges.Total'] = datos_cliente['account.Charges.Total'].replace(' ',pd.NA).fillna(0)

In [18]:
#Pasar a tipo float los datos de los cargos totales
datos_cliente['account.Charges.Total']=datos_cliente['account.Charges.Total'].astype(np.float64)

In [22]:
datos_cliente.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   customerID         7267 non-null   object 
 1   Still_Customer     7267 non-null   object 
 2   Gender             7267 non-null   object 
 3   Senior_Citizen     7267 non-null   int64  
 4   Partner            7267 non-null   object 
 5   Dependents         7267 non-null   object 
 6   Contract_Months    7267 non-null   int64  
 7   Phone_Service      7267 non-null   object 
 8   Multiple_Lines     7267 non-null   object 
 9   Internet_Service   7267 non-null   object 
 10  Online_Security    7267 non-null   object 
 11  Online_Backup      7267 non-null   object 
 12  Device_Protection  7267 non-null   object 
 13  Tech_Support       7267 non-null   object 
 14  Streaming_TV       7267 non-null   object 
 15  Streaming_Movies   7267 non-null   object 
 16  Contract_type      7267 

### Formatear nombres de columnas para mejor entendimiento en presentación

In [19]:
columnas=['customerID',
          'Still_Customer',
          'Gender',
          'Senior_Citizen',
          'Partner',
          'Dependents',
          'Contract_Months',
          'Phone_Service',
          'Multiple_Lines',
          'Internet_Service',
          'Online_Security',
          'Online_Backup',
          'Device_Protection',
          'Tech_Support',
          'Streaming_TV',
          'Streaming_Movies',
          'Contract_type',
          'Paperless_Billing',
          'Payment_Method',
          'Charges_Monthly',
          'Charges_Total']
datos_cliente.columns = columnas

Crear columa de cargos al día "Charges_Daily"

In [20]:
#
gastos_diarios=datos_cliente['Charges_Monthly'].to_numpy()
gastos_diarios = gastos_diarios/30
datos_cliente['Charges_Day'] = np.round(gastos_diarios,2)

In [21]:
datos_cliente.head()

,customerID,Still_Customer,Gender,Senior_Citizen,Partner,Dependents,Contract_Months,Phone_Service,Multiple_Lines,Internet_Service,...,Device_Protection,Tech_Support,Streaming_TV,Streaming_Movies,Contract_type,Paperless_Billing,Payment_Method,Charges_Monthly,Charges_Total,Charges_Day
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.30,2.19
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.40,2.00
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85,2.46
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85,3.27
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.40,2.80


Hay columnas en las cuales solo existen los valores unicos de "Yes" y "No" , por lo que se pueden cambiar valores a 0 y 1 para poder hacer operaciones mas sencillas, hay otros que tienen mas valores unicos que no se pueden cambiar tan facilmente, así que el análisis será distinto

Las siguientes columans son las que pueden cambiar a eso
 * 'Still_Customer'
 * 'Partner'
 * 'Dependents'
 * 'Phone_Service'
 * 'Paperless_Billing'

In [80]:
datos_cliente[['Still_Customer',
                'Partner',
                'Dependents',
                'Phone_Service',
                'Paperless_Billing']]=datos_cliente[['Still_Customer',
                                      'Partner',
                                      'Dependents',
                                      'Phone_Service',
                                      'Paperless_Billing']].replace('No',0).replace('Yes',1);
# cambiar el tipo de dato de estas columnas para poder trabajar con ellas como enteros
datos_cliente[['Still_Customer',
                'Partner',
                'Dependents',
                'Phone_Service',
                'Paperless_Billing']]\
                =datos_cliente[['Still_Customer',
                                'Partner',
                                'Dependents',
                                'Phone_Service',
                                'Paperless_Billing']].astype(int)

In [81]:
datos_cliente.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   customerID         7267 non-null   object 
 1   Still_Customer     7267 non-null   int64  
 2   Gender             7267 non-null   object 
 3   Senior_Citizen     7267 non-null   int64  
 4   Partner            7267 non-null   int64  
 5   Dependents         7267 non-null   int64  
 6   Contract_Months    7267 non-null   int64  
 7   Phone_Service      7267 non-null   int64  
 8   Multiple_Lines     7267 non-null   object 
 9   Internet_Service   7267 non-null   object 
 10  Online_Security    7267 non-null   object 
 11  Online_Backup      7267 non-null   object 
 12  Device_Protection  7267 non-null   object 
 13  Tech_Support       7267 non-null   object 
 14  Streaming_TV       7267 non-null   object 
 15  Streaming_Movies   7267 non-null   object 
 16  Contract_type      7267 

## Análisis descriptivo de valores

In [82]:
datos_cliente.describe()

,Still_Customer,Senior_Citizen,Partner,Dependents,Contract_Months,Phone_Service,Paperless_Billing,Charges_Monthly,Charges_Total,Charges_Day
count,7267.000000,7267.000000,7267.000000,7267.000000,7267.000000,7267.000000,7267.000000,7267.000000,7267.000000,7267.000000
mean,0.257190,0.162653,0.484106,0.300124,32.346498,0.902711,0.593230,64.720098,2277.182035,2.157292
std,0.437115,0.369074,0.499782,0.458343,24.571773,0.296371,0.491265,30.129572,2268.648587,1.004407
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,18.250000,0.000000,0.610000
25%,0.000000,0.000000,0.000000,0.000000,9.000000,1.000000,0.000000,35.425000,396.200000,1.180000
50%,0.000000,0.000000,0.000000,0.000000,29.000000,1.000000,1.000000,70.300000,1389.200000,2.340000
75%,1.000000,0.000000,1.000000,1.000000,55.000000,1.000000,1.000000,89.875000,3778.525000,2.995000
max,1.000000,1.000000,1.000000,1.000000,72.000000,1.000000,1.000000,118.750000,8684.800000,3.960000
